# 1. Load SQLite and Database
- 'CourseData.db'
- note the 3 backslashes

In [1]:
%reload_ext sql
%sql sqlite:///CourseData.db

'Connected: @CourseData.db'

In [2]:
%%sql
DROP TABLE IF EXISTS CourseData;

 * sqlite:///CourseData.db
Done.


[]

## DROP Tables 

In [3]:
%%sql
sqlite:///CourseData.db
DROP TABLE IF EXISTS COURSECATALOGS;
DROP TABLE IF EXISTS COURSES;
DROP TABLE IF EXISTS TEACHERS;
DROP TABLE IF EXISTS COURSEMEETINGS;

Done.
Done.
Done.
Done.


[]

# 2. Find COUNT of each .csv file

In [4]:
%%sql
SELECT Count(*) FROM import_course_meetings;

 * sqlite:///CourseData.db
Done.


Count(*)
317339


In [5]:
%%sql
SELECT Count(*) FROM import_courses;

 * sqlite:///CourseData.db
Done.


Count(*)
15955


In [6]:
%%sql
SELECT catalog_id FROM import_Course_Catalog limit 1;


 * sqlite:///CourseData.db
Done.


catalog_id
AN 0301


In [7]:
%%sql
SELECT Count(*) FROM import_Course_Catalog;

 * sqlite:///CourseData.db
Done.


Count(*)
4441


# 3. Create Tables
- primary key is generated on every table by naming new surrogate key/field that isn't already in .csv files.
- don't need NOT NULL for PK because it's implied in SQLite.


In [8]:
%%sql
CREATE TABLE COURSECATALOGS (
    CCID INTEGER PRIMARY KEY,
    catalog_id TEXT,
    program_code TEXT,    
    program_name TEXT,
    course_title TEXT,
    prereqs TEXT,    
    coreqs TEXT,
    fees TEXT,
    attributes TEXT,    
    description TEXT
);

 * sqlite:///CourseData.db
Done.


[]

In [9]:
%%sql
CREATE TABLE TEACHERS (
 TID INTEGER PRIMARY KEY,
 primary_instructor TEXT NOT NULL
);

 * sqlite:///CourseData.db
Done.


[]

In [10]:
%%sql
CREATE TABLE COURSES (
                 crn SMALLINT,
                 term VARCAR(20),
                 catalog_id TEXT,
                 section TEXT,
                 credits TEXT,
                 title TEXT,
                 meetings TEXT,
                 timecodes TEXT,
                 cap TEXT,
                 act TEXT,
                 rem TEXT,
                 TID INTEGER,
                 PRIMARY KEY(crn,term)
);

 * sqlite:///CourseData.db
Done.


[]

In [11]:
%%sql
CREATE TABLE COURSEMEETINGS (
    
    CID INTEGER PRIMARY KEY,
    crn SMALLINT,
    term VARCHAR(20),
    location TEXT NOT NULL,
    day TEXT NOT NULL,
    start TIME NOT NULL,
    end TIME NOT NULL  
);

 * sqlite:///CourseData.db
Done.


[]

# 4. Do INSERT Statements

In [12]:
%%sql
INSERT INTO COURSECATALOGS (catalog_id, program_code, program_name, course_title, prereqs, coreqs, fees, attributes, description) 
SELECT DISTINCT catalog_id, program_code, program_name, course_title, prereqs, coreqs, fees, attributes, description
FROM import_Course_Catalog;

 * sqlite:///CourseData.db
2221 rows affected.


[]

In [13]:
%%sql
SELECT * FROM COURSECATALOGS
limit 2;

 * sqlite:///CourseData.db
Done.


CCID,catalog_id,program_code,program_name,course_title,prereqs,coreqs,fees,attributes,description
1,AN 0301,AN,Asian Studies,Independent Study,,,,,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.
2,AN 0310,AN,Asian Studies,Asian Studies Seminar,,,,,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only."


# INSERT TEACHERS

In [14]:
%%sql
DELETE FROM TEACHERS;

INSERT INTO TEACHERS  (primary_instructor)
SELECT DISTINCT primary_instructor
    FROM import_courses
    ;

 * sqlite:///CourseData.db
0 rows affected.
1105 rows affected.


[]

In [15]:
%%sql
SELECT * FROM TEACHERS
limit 2;

 * sqlite:///CourseData.db
Done.


TID,primary_instructor
1,Michael P. Coyne
2,Rebecca I. Bloch


# INSERT COURSEMEETINGS

In [16]:
%%sql 
INSERT INTO COURSEMEETINGS ( crn, term, location, day, start, end)
SELECT DISTINCT crn, term, location, day, start, end
FROM import_course_meetings


 * sqlite:///CourseData.db
311142 rows affected.


[]

In [17]:
%%sql
SELECT * FROM COURSEMEETINGS
LIMIT 10;

 * sqlite:///CourseData.db
Done.


CID,crn,term,location,day,start,end
1,70384,Fall2014,DSB 105,T,2014-09-02T08:00:00,2014-09-02T09:15:00
2,70384,Fall2014,DSB 105,F,2014-09-05T08:00:00,2014-09-05T09:15:00
3,70384,Fall2014,DSB 105,T,2014-09-09T08:00:00,2014-09-09T09:15:00
4,70384,Fall2014,DSB 105,F,2014-09-12T08:00:00,2014-09-12T09:15:00
5,70384,Fall2014,DSB 105,T,2014-09-16T08:00:00,2014-09-16T09:15:00
6,70384,Fall2014,DSB 105,F,2014-09-19T08:00:00,2014-09-19T09:15:00
7,70384,Fall2014,DSB 105,T,2014-09-23T08:00:00,2014-09-23T09:15:00
8,70384,Fall2014,DSB 105,F,2014-09-26T08:00:00,2014-09-26T09:15:00
9,70384,Fall2014,DSB 105,T,2014-09-30T08:00:00,2014-09-30T09:15:00
10,70384,Fall2014,DSB 105,F,2014-10-03T08:00:00,2014-10-03T09:15:00


# INSERT COURSES
- _key:_ do NOT need primary_instructor in courses table since we're going to join in the teachers table that has this value.  We are simply telling it what to JOIN these tables on, but primary_instructor does not need to be in both tables.

- still need to add additional FKs

In [18]:
%%sql
INSERT INTO COURSES (crn, term,catalog_id, section, credits, title, meetings, timecodes, cap, act, rem, TID)
SELECT DISTINCT crn, term, catalog_id, section, credits, title, meetings, timecodes, cap, act, rem, TID
FROM import_courses
JOIN TEACHERS ON TEACHERS.primary_instructor = import_courses.primary_instructor;


 * sqlite:///CourseData.db
15938 rows affected.


[]

In [19]:
%%sql
SELECT * FROM COURSES
LIMIT 5;

 * sqlite:///CourseData.db
Done.


crn,term,catalog_id,section,credits,title,meetings,timecodes,cap,act,rem,TID
70384,Fall2014,AC 0011,C01,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],0,31,-31,1
70385,Fall2014,AC 0011,C02,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0930am-1045am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0930am-1045am 09/02-12/08 DSB 105'],0,31,-31,1
70382,Fall2014,AC 0011,C03,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 1230pm-0145pm 09/02-12/08 DSB 105'],0,31,-31,1
70291,Fall2014,AC 0011,C04,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '1100am-1215pm', 'dates': '09/02-12/08', 'location': 'DSB 111'}]",['MR 1100am-1215pm 09/02-12/08 DSB 111'],0,29,-29,2
70350,Fall2014,AC 0011,C05,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 111'}]",['MR 1230pm-0145pm 09/02-12/08 DSB 111'],0,30,-30,2


In [20]:
%%sql
SELECT catalog_id, primary_instructor 
FROM COURSES
JOIN TEACHERS USING (TID)
limit 10;

 * sqlite:///CourseData.db
Done.


catalog_id,primary_instructor
AC 0011,Michael P. Coyne
AC 0011,Michael P. Coyne
AC 0011,Michael P. Coyne
AC 0011,Rebecca I. Bloch
AC 0011,Rebecca I. Bloch
AC 0011,Rebecca I. Bloch
AC 0011,Paul Caster
AC 0011,Jo Ann Drusbosky
AC 0011,Jo Ann Drusbosky
AC 0011,Jo Ann Drusbosky


In [21]:
%%sql
SELECT * FROM COURSES
limit 5
;


 * sqlite:///CourseData.db
Done.


crn,term,catalog_id,section,credits,title,meetings,timecodes,cap,act,rem,TID
70384,Fall2014,AC 0011,C01,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],0,31,-31,1
70385,Fall2014,AC 0011,C02,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0930am-1045am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0930am-1045am 09/02-12/08 DSB 105'],0,31,-31,1
70382,Fall2014,AC 0011,C03,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 1230pm-0145pm 09/02-12/08 DSB 105'],0,31,-31,1
70291,Fall2014,AC 0011,C04,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '1100am-1215pm', 'dates': '09/02-12/08', 'location': 'DSB 111'}]",['MR 1100am-1215pm 09/02-12/08 DSB 111'],0,29,-29,2
70350,Fall2014,AC 0011,C05,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 111'}]",['MR 1230pm-0145pm 09/02-12/08 DSB 111'],0,30,-30,2


In [22]:
%%sql
SELECT TID FROM TEACHERS
limit 5;

 * sqlite:///CourseData.db
Done.


TID
1
2
3
4
5


# 5. Run SELECT Queries

In [23]:
%reload_ext sql
%sql sqlite:///CourseData.db

'Connected: @CourseData.db'

## SELECT Query for all unique classrooms 
- only included those with a character lenght of 7 (some had 2, 8, etc.)
- can't figure out how to get a distinct location when including additinonal fields.

In [24]:
%%sql
SELECT DISTINCT 
location
FROM COURSEMEETINGS
WHERE LENGTH(location) = 7
ORDER BY location
LIMIT 20
;

 * sqlite:///CourseData.db
Done.


location
BCC 200
BLM 112
BNW 124
BNW 127
BNW 128
BNW 130
BNW 131
BNW 133
BNW 136
BNW 137


## SELECT Query displaying all courses in MSBA Program '18-'19
- still have to get rid of blank by making defining description as NOT NULL.
- is there an easier way?

In [39]:
%%sql
SELECT course_title AS Course, program_name AS Program, catalog_id AS Code, description AS Description
FROM COURSECATALOGS
WHERE program_name = 'Information Systems'
AND
Code LIKE "IS 05%"
OR
Code LIKE "QA 05%"
OR
Code LIKE "SW 0518%"
ORDER BY program_name
;

 * sqlite:///CourseData.db
Done.


Course,Program,Code,Description
Information Systems and Database Management,Information Systems,IS 0500,"This course introduces the basic concepts and tools relevant to information systems and database management, and their enabling roles in business strategies and operations. Case studies are used to facilitate discussions of practical applications and issues involving strategic alignments of organizations, resource allocation, integration, planning, and analysis of cost, benefit and performance in light of the big data challenges. Specific emphases involve database design and implementation and emerging strategies and technologies such as business intelligence, big data management, web security, and online business analytics."
International Information Systems,Information Systems,IS 0501,"This course examines information technology environments around the world, and attendant challenges to business strategy and information systems design. The course identifies geographic and institutional variables that create borders in the global Internet economy: material infrastructures, socio-economic elements, and political-legal systems. The course emphasizes national and regional strategies, emergent technologies, hybrid systems, and equity issues."
Python for Business Analytics,Information Systems,IS 0505,"In this course, we introduce Python as a language and tool for collecting, preprocessing, and visualizing data for business analytics. since Python is one of the most popular programming languages, along with R, in data mining and business analytics, its fundamental programming logic and knowledge is essential for students to apply in data mining and to succeed in the job market. Specifically, this course focuses on the data-engineering phase, which includes collecting, preprocessing, and visualizing data, with respect to applications in business modeling, optimization, and statistical analysis. In addition, a number of mini projects will be used as vehicles to cover the main applications of data analytics, including recommender systems, text analytics, and web analytics."
Databases for Business Analytics,Information Systems,IS 0510,"This course introduces databases and data management in three parts. The first part covers basic database fundamentals. The second part is a hands-on introduction to Structured Query Language (SQL) for defining, manipulating, accessing, and managing data, accompanied by the basics of data modeling and normalization needed to ensure data integrity. The course concludes with a comprehensive database project that gives each student the opportunity to integrate and apply the new knowledge and skills learned from this class. Advanced topics such as distributed database systems, data services, and NoSQL databases are also discussed."
Project Management,Information Systems,IS 0520,"This course explores the process and practice of project management. Topics to be covered include project lifecycle and organizations, teambuilding and productivity, task scheduling and resource allocation, and progress tracking and control. Cases will be used to consider the implications for change management, consulting, IT implementation, and other related disciplines. Small team projects and experiential exercises will also be used to provide an active learning environment. This course is designed to count toward professional project management certification."
Data Mining and Business Intelligence,Information Systems,IS 0540,"This course will change the way you think about data and its role in business. Businesses, governments, and individuals create massive collections of data as a byproduct of their activity. Increasingly, managers rely on intelligent technology to systematically analyze data to improve their decision-making. In many cases, automating analytical and decision-making processes is necessary because of the large volume of data and the speed with which new data are generated. In this course, we will examine how d

# JOIN Testing

In [26]:
%%sql
SELECT *
FROM COURSES
LIMIT 7;

 * sqlite:///CourseData.db
Done.


crn,term,catalog_id,section,credits,title,meetings,timecodes,cap,act,rem,TID
70384,Fall2014,AC 0011,C01,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],0,31,-31,1
70385,Fall2014,AC 0011,C02,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0930am-1045am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0930am-1045am 09/02-12/08 DSB 105'],0,31,-31,1
70382,Fall2014,AC 0011,C03,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 1230pm-0145pm 09/02-12/08 DSB 105'],0,31,-31,1
70291,Fall2014,AC 0011,C04,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '1100am-1215pm', 'dates': '09/02-12/08', 'location': 'DSB 111'}]",['MR 1100am-1215pm 09/02-12/08 DSB 111'],0,29,-29,2
70350,Fall2014,AC 0011,C05,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 111'}]",['MR 1230pm-0145pm 09/02-12/08 DSB 111'],0,30,-30,2
70381,Fall2014,AC 0011,C06,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '0330pm-0445pm', 'dates': '09/02-12/08', 'location': 'DSB 110A'}]",['MR 0330pm-0445pm 09/02-12/08 DSB 110A'],0,31,-31,2
70383,Fall2014,AC 0011,G,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '0930am-1045am', 'dates': '09/02-12/08', 'location': 'DSB 111'}]",['MR 0930am-1045am 09/02-12/08 DSB 111'],30,31,-1,3


%%sql
SELECT t.TID, t.primary_instructor, c.catalog_id
FROM COURSES as c
JOIN TEACHERS as t ON c.TID = t.TID
LIMIT 100;

# Dr. Huntley
### What Courses has Dr. Huntley Taught?

In [27]:
%%sql
SELECT t.primary_instructor, c.catalog_id
FROM COURSES as c
JOIN TEACHERS as t ON c.TID = t.TID
WHERE primary_instructor like "Christopher L.%"
Limit 10;

 * sqlite:///CourseData.db
Done.


primary_instructor,catalog_id
Christopher L. Huntley,IS 0100
Christopher L. Huntley,IS 0135
Christopher L. Huntley,IS 0320
Christopher L. Huntley,IS 0100
Christopher L. Huntley,IS 0100
Christopher L. Huntley,IS 0135
Christopher L. Huntley,IS 0320
Christopher L. Huntley,IS 0135
Christopher L. Huntley,IS 0240
Christopher L. Huntley,IS 0585C


### How Many Courses has Dr. Huntley Taught?

In [28]:
%%sql
SELECT DISTINCT t.primary_instructor, COUNT(c.catalog_id) AS courses_taught
FROM COURSES as c
JOIN TEACHERS as t ON c.TID = t.TID
WHERE primary_instructor like "Christopher L.%"
Limit 10;

 * sqlite:///CourseData.db
Done.


primary_instructor,courses_taught
Christopher L. Huntley,31


### What Courses has Dr. Massey Taught?

In [29]:
%%sql
SELECT t.primary_instructor, c.catalog_id
FROM COURSES as c
JOIN TEACHERS as t ON c.TID = t.TID
WHERE primary_instructor like "Dawn W%"
Limit 10;

 * sqlite:///CourseData.db
Done.


primary_instructor,catalog_id
Dawn W Massey,AC 0570
Dawn W Massey,AC 0590
Dawn W Massey,AC 0590
Dawn W Massey,AC 0590
Dawn W Massey,AC 0570
Dawn W Massey,AC 0590
Dawn W Massey,AC 0590
Dawn W Massey,AC 0590
Dawn W Massey,AC 0570
Dawn W Massey,AC 0570


### How Many Courses has Dr. Massey Taught?

In [30]:
%%sql
SELECT DISTINCT t.primary_instructor, COUNT(c.catalog_id) AS courses_taught
FROM COURSES as c
JOIN TEACHERS as t ON c.TID = t.TID
WHERE primary_instructor like "Dawn%"
Limit 10;

 * sqlite:///CourseData.db
Done.


primary_instructor,courses_taught
Dawn W Massey,52


In [31]:
%%sql
SELECT TID, primary_instructor FROM TEACHERS
limit 10;


 * sqlite:///CourseData.db
Done.


TID,primary_instructor
1,Michael P. Coyne
2,Rebecca I. Bloch
3,Paul Caster
4,Jo Ann Drusbosky
5,Arleen N. Kardos
6,Scott M Brenner
7,Kevin C. Cassidy
8,Bruce Bradford
9,Milo W. Peck
10,Stephen E. Yost


In [32]:
%%sql
SELECT c.term, cm.start, cm.location, c.catalog_id
FROM COURSES AS c
JOIN COURSEMEETINGS AS cm
LIMIT 20;

 * sqlite:///CourseData.db
Done.


term,start,location,catalog_id
Fall2014,2014-09-02T08:00:00,DSB 105,AC 0011
Fall2014,2014-09-05T08:00:00,DSB 105,AC 0011
Fall2014,2014-09-09T08:00:00,DSB 105,AC 0011
Fall2014,2014-09-12T08:00:00,DSB 105,AC 0011
Fall2014,2014-09-16T08:00:00,DSB 105,AC 0011
Fall2014,2014-09-19T08:00:00,DSB 105,AC 0011
Fall2014,2014-09-23T08:00:00,DSB 105,AC 0011
Fall2014,2014-09-26T08:00:00,DSB 105,AC 0011
Fall2014,2014-09-30T08:00:00,DSB 105,AC 0011
Fall2014,2014-10-03T08:00:00,DSB 105,AC 0011


In [33]:
%%sql
SELECT cm.start, c.catalog_id
FROM COURSES AS c
JOIN COURSEMEETINGS AS cm
LIMIT 10;

 * sqlite:///CourseData.db
Done.


start,catalog_id
2014-09-02T08:00:00,AC 0011
2014-09-05T08:00:00,AC 0011
2014-09-09T08:00:00,AC 0011
2014-09-12T08:00:00,AC 0011
2014-09-16T08:00:00,AC 0011
2014-09-19T08:00:00,AC 0011
2014-09-23T08:00:00,AC 0011
2014-09-26T08:00:00,AC 0011
2014-09-30T08:00:00,AC 0011
2014-10-03T08:00:00,AC 0011


Have to finish insterting COURSES column

In [34]:
%%sql
SELECT TID, primary_instructor FROM TEACHERS
limit 10;

 * sqlite:///CourseData.db
Done.


TID,primary_instructor
1,Michael P. Coyne
2,Rebecca I. Bloch
3,Paul Caster
4,Jo Ann Drusbosky
5,Arleen N. Kardos
6,Scott M Brenner
7,Kevin C. Cassidy
8,Bruce Bradford
9,Milo W. Peck
10,Stephen E. Yost


In [35]:
%%sql
SELECT course_title AS Course,  catalog_id AS Code, program_name AS Program, 
FROM COURSECATALOGS
WHERE credits = "6 credits"
LIMIT 50;

 * sqlite:///CourseData.db
(sqlite3.OperationalError) near "FROM": syntax error [SQL: 'SELECT course_title AS Course,  catalog_id AS Code, program_name AS Program, \nFROM COURSECATALOGS\nWHERE credits = "6 credits"\nLIMIT 50;'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [36]:
%%sql
VACUUM;

 * sqlite:///CourseData.db
Done.


[]